# Study note: Ontologies
_[Benedikt Venn](https://github.com/bvenn), Feb 2023_

## What is an ontology?
_An ontology is a formal specification of the terms in the domain (taxonomy) and relations among them_. It defines a common vocabulary for researchers. These can be used by domain experts to share and annotate information in their fields, to categorize observations, and to extract and aggregate information from a huge data sets in an semi-automated manner.
The main components of an ontology are **classes** - the entities within the domain - and **roles** which are the properties of the classes. To fill this abstract definition of an ontology with something tangible, in the following an ontology for bioentities is generated.

A **class** of _Protein_ represent all proteins. Cytochrom c oxidase (COX) is an **instance** of the _Protein_ class. A class can have several subclasses, e.g. _Membrane protein_/_Globular protein_ or _Enzymatic_/_Structural_ etc. that are more specific than the superclass. Consequently every instance of class _Membrane protein_ also is an instance of class '_Protein_. **Roles** describe properties of the single instances. A protein e.g. could have the roles (i) _mass_, (ii) _sequence_, (iii) _localization_, and (iv) _full name_. These are properties of all instances of the _Protein_ class. The _membrane protein_ subclass may have additional properties, that are specific to this kind of protein, e.g. _number of transmembrane domains_. The value types of the roles could be primitive types (string, int, float), or instances of another class. _Compartment_ for example could be the value of field _localization_. _Compartment_ itself could have several roles and relationships.

<img src="../../img/ontologies/ontology_venn_01.png"></img>

Fig. 1: Some classes, instances and relations of the cell domain. The interdependence (inverse relation) of COX and Mitochondrion (blue arrows) is not mandatory.

<p style="text-align: center;"></p>

### Role cardinality
The cardinality defines how many values a role can have. The mass is distinct and the same for a single protein indicating a single cardinality, while present-in is a multi-cardinality role of the _Compartment_ class. Sometimes is can necessary to set the cardinality of a subclass role to zero.
If an instance can be assigned to more than one ‘class branch’ an additional class can be defined, that has several superclasses. Consequently it inherits the roles from all its superclasses.


### Design rules
-	If B is a subclass of A and C is a subclass of B, then C is a subclass of A
-	Synonyms for the same concept do not represent different classes
-	Avoid class cycles (C is a B, B is a A, and A is a C)
-	All siblings must be at the same level of generality (e.g. children of proteins should **not** be `structural proteins` and `small HSPs`)
-	If a class has only one subclass, consider remodeling or completing the ontology
-	If a class has more than a dozen subclasses, consider intermediate categories

## OBO
The [Open Biological and Biomedical Ontologies (OBO) Foundry](obofoundry.org) is a collection of ontologies relevant for life sciences. Additionally, obo defines a file format, that unifies the distribution of ontologies that is based on the principles of the Web Ontology Language (OWL).

### Gene Ontology
The GeneOntology (GO) provides a uniform way to describe the functions of gene products from organisms across all kingdoms of life and thereby enable analysis of genomic data. It can be found [here]( http://current.geneontology.org/ontology/go.obo).

Example of a GO class in OBO format:

```
[Term]
id: GO:0000269
name: toxin export channel activity
namespace: molecular_function
def: "Enables the energy independent passage of toxins, sized less than 1000 Da, across a membrane towards the outside of the cell. The transmembrane portions of porins consist exclusively of beta-strands which form a beta-barrel. They are found in the outer membranes of Gram-negative bacteria, mitochondria, plastids and possibly acid-fast Gram-positive bacteria." [GOC:mtg_transport]
is_a: GO:0015288 ! porin activity
is_a: GO:0019534 ! toxin transmembrane transporter activity
```

A class is initialized with the `[Term]` keyword. Roles are separated from its corresponding values by a colon. A term can be accessed by its unique _id_ that points to this entry. You can see two is_a relationships of ‘toxin export channel activity’. Thereby it is clear that it represents a subclass of the two superclasses `porin activity` and `toxin transmembrane transporter activity`. If a protein is assigned to GO term 0000269 it automatically is a children of GO:0015288 and GO:0019534.

## References
1. Noy and McGuinnes, Ontology Development 101: A Guide to Creating Your First Ontology, Stanford Knowledge Systems Laboratory Technical Report, 2001
2. [The Open Biological and Biomedical Ontology](https://obofoundry.org/)
3. Ashburner et al., Gene ontology: tool for the unification of biology, Nat Genet, 2000, [doi:10.1038/75556](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3037419/)

## Practice

The goal is to visualize a subbranch of the gene ontology using [BioFSharp.IO](https://github.com/CSBiology/BioFSharp) for parsing and [Cyjs.NET](https://github.com/fslaborg/Cyjs.NET) for visualization. Parsing describes the process of analyzing a text according to a defined set of rules (e.g. the OBO file format convention).

### Task 1
- read in the GO ontology as `string array` 



In [2]:
#r "nuget: FSharp.Data"
#r "nuget: BioFSharp.IO, 2.0.0-preview.3"
#r "nuget: Cyjs.NET, 0.0.4"

open FSharp.Data    //required for data download 
open BioFSharp.IO   //required for parsing
open Cyjs.NET       //required for visualization

let getRawString url = 
    Http.RequestString url
    |> fun x -> x.Split '\n'

let goUrl = @"http://current.geneontology.org/ontology/go.obo"

let goRawData = getRawString goUrl


Installed Packages BioFSharp.IO, 2.0.0-preview.3 Cyjs.NET, 0.0.4 FSharp.Data, 5.0.2

### Task 2 
- parse the data using the OBO parser of ´BioFSharp.IO´
  - set the verbosity parameter to `false` to 
  - convert the `seq<OboTerm>` into a `Oboterm []` after parsing

In [3]:
let go = 
    BioFSharp.IO.Obo.parseOboTerms false goRawData  
    |> Seq.toArray                                  


### Task 3
- filter obo terms that have `GO:0071704` as `id`, `relationship` or `is_A` relationship and call the filtered triple sequence goElements


In [4]:

let getElements (terms:Obo.OboTerm[]) id = 
    terms
    |> Array.filter (fun x -> 
        x.Id = id ||
        List.exists (fun (rel : string) -> rel.Contains id) x.IsA
        )
    |> Array.map (fun x -> 
        let source = x.Id
        let label = x.Name
        //sometimes a relationship can be stored as dedicated relationship rather than a isA role
        let target = List.append x.IsA x.Relationships
        target 
        |> List.map (fun t -> 
            let singleTarget = t.Split ' ' |> Seq.head
            label,source, singleTarget
            )
        )
    |> Seq.concat

let goElements = getElements go "GO:0071704" 
goElements

index,Item1,Item2,Item3
0,carbohydrate metabolic process,GO:0005975,GO:0044238
1,carbohydrate metabolic process,GO:0005975,GO:0071704
2,cellular aldehyde metabolic process,GO:0006081,GO:0044237
3,cellular aldehyde metabolic process,GO:0006081,GO:0071704
4,organic acid metabolic process,GO:0006082,GO:0044237
5,organic acid metabolic process,GO:0006082,GO:0044281
6,organic acid metabolic process,GO:0006082,GO:0071704
7,lipid metabolic process,GO:0006629,GO:0044238
8,lipid metabolic process,GO:0006629,GO:0071704
9,glutathione metabolic process,GO:0006749,GO:0071704


### Task 4
- create vertices and edges

In [5]:

let getCytoVertices (input: seq<string*string*string>) = 
    input
    |> Seq.collect (fun (l,s,t) ->
        let stylingSource = [CyParam.label (sprintf "%s %s" l s); CyParam.weight 24]
        let stylingTarget = [CyParam.label (sprintf "%s %s" l t); CyParam.weight 24]
        [|Elements.node s stylingSource;Elements.node t stylingTarget|]
        )
    |> Seq.distinct

let getCytoEdges (input: seq<string*string*string>)= 
    input 
    |> Seq.distinct
    |> Seq.mapi (fun i (l,s,t) -> 
        let styling = [CyParam.weight 0.3]
        Elements.edge ("e" + string i) s t styling
        )

let goVertices = getCytoVertices goElements
let goEdges = getCytoEdges goElements

### Task 5
- create and show CyGraph
- is there any parent of `GO:0071704`? 
  - Answer: ________

In [13]:
let cytoGraph vertices edges = 
    CyGraph.initEmpty ()
    |> CyGraph.withElements vertices
    |> CyGraph.withElements edges
    |> CyGraph.withStyle "node" 
        [
            CyParam.shape "circle"
            CyParam.content =. CyParam.label
            CyParam.Border.color "#A00975"
        ]
    |> CyGraph.withStyle "edge" 
        [
            CyParam.Line.color "#3D1244"
            CyParam.Curve.style "bezier"
            CyParam.Target.Arrow.shape "triangle"
        ]
    |> CyGraph.withLayout (Layout.initBreadthfirst id)  

// Send the cytograph to the browser
cytoGraph goVertices goEdges
|> CyGraph.withSize (1300,1000)



<html>
<head>
<script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min.js"></script>
</head>
<body> <style>#e89e1de76a3 { width: 1300px; height: 1000px; display: block } </style>
<div id="e89e1de76a3"></div>
<script type="text/javascript">
            var renderCyjs_d756597165f44fed9cebeea2b212d80d = function() {
            var fsharpCyjsRequire = requirejs.config({context:'fsharp-cyjs',paths:{cyjs:'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min'}}) || require;
            fsharpCyjsRequire(['cyjs'], function(Cyjs) {
            var graphdata = {"container":document.getElementById('e89e1de76a3'),"elements":[{"data":{"id":"GO:0005975","label":"carbohydrate metabolic process GO:0005975","weight":24}},{"data":{"id":"GO:0044238","label":"carbohydrate metabolic process GO:0044238","weight":24}},{"data":{"id":"GO:0005975","label":"carbohydrate metabolic process GO:0005975","weight":24}},{"data":{"id":"GO:0071704","label":"carbohydrate metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0006081","label":"cellular aldehyde metabolic process GO:0006081","weight":24}},{"data":{"id":"GO:0044237","label":"cellular aldehyde metabolic process GO:0044237","weight":24}},{"data":{"id":"GO:0006081","label":"cellular aldehyde metabolic process GO:0006081","weight":24}},{"data":{"id":"GO:0071704","label":"cellular aldehyde metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0006082","label":"organic acid metabolic process GO:0006082","weight":24}},{"data":{"id":"GO:0044237","label":"organic acid metabolic process GO:0044237","weight":24}},{"data":{"id":"GO:0006082","label":"organic acid metabolic process GO:0006082","weight":24}},{"data":{"id":"GO:0044281","label":"organic acid metabolic process GO:0044281","weight":24}},{"data":{"id":"GO:0006082","label":"organic acid metabolic process GO:0006082","weight":24}},{"data":{"id":"GO:0071704","label":"organic acid metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0006629","label":"lipid metabolic process GO:0006629","weight":24}},{"data":{"id":"GO:0044238","label":"lipid metabolic process GO:0044238","weight":24}},{"data":{"id":"GO:0006629","label":"lipid metabolic process GO:0006629","weight":24}},{"data":{"id":"GO:0071704","label":"lipid metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0006749","label":"glutathione metabolic process GO:0006749","weight":24}},{"data":{"id":"GO:0071704","label":"glutathione metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0009812","label":"flavonoid metabolic process GO:0009812","weight":24}},{"data":{"id":"GO:0071704","label":"flavonoid metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0015977","label":"carbon fixation GO:0015977","weight":24}},{"data":{"id":"GO:0071704","label":"carbon fixation GO:0071704","weight":24}},{"data":{"id":"GO:0018904","label":"ether metabolic process GO:0018904","weight":24}},{"data":{"id":"GO:0044281","label":"ether metabolic process GO:0044281","weight":24}},{"data":{"id":"GO:0018904","label":"ether metabolic process GO:0018904","weight":24}},{"data":{"id":"GO:0071704","label":"ether metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018907","label":"dimethyl sulfoxide metabolic process GO:0018907","weight":24}},{"data":{"id":"GO:0006790","label":"dimethyl sulfoxide metabolic process GO:0006790","weight":24}},{"data":{"id":"GO:0018907","label":"dimethyl sulfoxide metabolic process GO:0018907","weight":24}},{"data":{"id":"GO:0071704","label":"dimethyl sulfoxide metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018937","label":"nitroglycerin metabolic process GO:0018937","weight":24}},{"data":{"id":"GO:0006805","label":"nitroglycerin metabolic process GO:0006805","weight":24}},{"data":{"id":"GO:0018937","label":"nitroglycerin metabolic process GO:0018937","weight":24}},{"data":{"id":"GO:0006807","label":"nitroglycerin metabolic process GO:0006807","weight":24}},{"data":{"id":"GO:0018937","label":"nitroglycerin metabolic process GO:0018937","weight":24}},{"data":{"id":"GO:0071704","label":"nitroglycerin metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018938","label":"2-nitropropane metabolic process GO:0018938","weight":24}},{"data":{"id":"GO:0006805","label":"2-nitropropane metabolic process GO:0006805","weight":24}},{"data":{"id":"GO:0018938","label":"2-nitropropane metabolic process GO:0018938","weight":24}},{"data":{"id":"GO:0034641","label":"2-nitropropane metabolic process GO:0034641","weight":24}},{"data":{"id":"GO:0018938","label":"2-nitropropane metabolic process GO:0018938","weight":24}},{"data":{"id":"GO:0071704","label":"2-nitropropane metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018942","label":"organometal metabolic process GO:0018942","weight":24}},{"data":{"id":"GO:0044237","label":"organometal metabolic process GO:0044237","weight":24}},{"data":{"id":"GO:0018942","label":"organometal metabolic process GO:0018942","weight":24}},{"data":{"id":"GO:0071704","label":"organometal metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018945","label":"organosilicon metabolic process GO:0018945","weight":24}},{"data":{"id":"GO:0006805","label":"organosilicon metabolic process GO:0006805","weight":24}},{"data":{"id":"GO:0018945","label":"organosilicon metabolic process GO:0018945","weight":24}},{"data":{"id":"GO:0071704","label":"organosilicon metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0018954","label":"pentaerythritol tetranitrate metabolic process GO:0018954","weight":24}},{"data":{"id":"GO:0006805","label":"pentaerythritol tetranitrate metabolic process GO:0006805","weight":24}},{"data":{"id":"GO:0018954","label":"pentaerythritol tetranitrate metabolic process GO:0018954","weight":24}},{"data":{"id":"GO:0006807","label":"pentaerythritol tetranitrate metabolic process GO:0006807","weight":24}},{"data":{"id":"GO:0018954","label":"pentaerythritol tetranitrate metabolic process GO:0018954","weight":24}},{"data":{"id":"GO:0071704","label":"pentaerythritol tetranitrate metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0019330","label":"aldoxime metabolic process GO:0019330","weight":24}},{"data":{"id":"GO:0006807","label":"aldoxime metabolic process GO:0006807","weight":24}},{"data":{"id":"GO:0019330","label":"aldoxime metabolic process GO:0019330","weight":24}},{"data":{"id":"GO:0071704","label":"aldoxime metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0019637","label":"organophosphate metabolic process GO:0019637","weight":24}},{"data":{"id":"GO:0006793","label":"organophosphate metabolic process GO:0006793","weight":24}},{"data":{"id":"GO:0019637","label":"organophosphate metabolic process GO:0019637","weight":24}},{"data":{"id":"GO:0071704","label":"organophosphate metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0030638","label":"polyketide metabolic process GO:0030638","weight":24}},{"data":{"id":"GO:0019748","label":"polyketide metabolic process GO:0019748","weight":24}},{"data":{"id":"GO:0030638","label":"polyketide metabolic process GO:0030638","weight":24}},{"data":{"id":"GO:0071704","label":"polyketide metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0035383","label":"thioester metabolic process GO:0035383","weight":24}},{"data":{"id":"GO:0006790","label":"thioester metabolic process GO:0006790","weight":24}},{"data":{"id":"GO:0035383","label":"thioester metabolic process GO:0035383","weight":24}},{"data":{"id":"GO:0071704","label":"thioester metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0042180","label":"cellular ketone metabolic process GO:0042180","weight":24}},{"data":{"id":"GO:0044237","label":"cellular ketone metabolic process GO:0044237","weight":24}},{"data":{"id":"GO:0042180","label":"cellular ketone metabolic process GO:0042180","weight":24}},{"data":{"id":"GO:0044281","label":"cellular ketone metabolic process GO:0044281","weight":24}},{"data":{"id":"GO:0042180","label":"cellular ketone metabolic process GO:0042180","weight":24}},{"data":{"id":"GO:0071704","label":"cellular ketone metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0042197","label":"halogenated hydrocarbon metabolic process GO:0042197","weight":24}},{"data":{"id":"GO:0006805","label":"halogenated hydrocarbon metabolic process GO:0006805","weight":24}},{"data":{"id":"GO:0042197","label":"halogenated hydrocarbon metabolic process GO:0042197","weight":24}},{"data":{"id":"GO:0071704","label":"halogenated hydrocarbon metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0042620","label":"poly(3-hydroxyalkanoate) metabolic process GO:0042620","weight":24}},{"data":{"id":"GO:0071704","label":"poly(3-hydroxyalkanoate) metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0042743","label":"hydrogen peroxide metabolic process GO:0042743","weight":24}},{"data":{"id":"GO:0071704","label":"hydrogen peroxide metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0042743","label":"hydrogen peroxide metabolic process GO:0042743","weight":24}},{"data":{"id":"GO:0072593","label":"hydrogen peroxide metabolic process GO:0072593","weight":24}},{"data":{"id":"GO:0043170","label":"macromolecule metabolic process GO:0043170","weight":24}},{"data":{"id":"GO:0071704","label":"macromolecule metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0046500","label":"S-adenosylmethionine metabolic process GO:0046500","weight":24}},{"data":{"id":"GO:0006790","label":"S-adenosylmethionine metabolic process GO:0006790","weight":24}},{"data":{"id":"GO:0046500","label":"S-adenosylmethionine metabolic process GO:0046500","weight":24}},{"data":{"id":"GO:0071704","label":"S-adenosylmethionine metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0050993","label":"dimethylallyl diphosphate metabolic process GO:0050993","weight":24}},{"data":{"id":"GO:0006793","label":"dimethylallyl diphosphate metabolic process GO:0006793","weight":24}},{"data":{"id":"GO:0050993","label":"dimethylallyl diphosphate metabolic process GO:0050993","weight":24}},{"data":{"id":"GO:0071704","label":"dimethylallyl diphosphate metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0071704","label":"organic substance metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0008152","label":"organic substance metabolic process GO:0008152","weight":24}},{"data":{"id":"GO:0120246","label":"acetylenic compound metabolic process GO:0120246","weight":24}},{"data":{"id":"GO:0071704","label":"acetylenic compound metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0120252","label":"hydrocarbon metabolic process GO:0120252","weight":24}},{"data":{"id":"GO:0071704","label":"hydrocarbon metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:0120254","label":"olefinic compound metabolic process GO:0120254","weight":24}},{"data":{"id":"GO:0071704","label":"olefinic compound metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1900558","label":"austinol metabolic process GO:1900558","weight":24}},{"data":{"id":"GO:0071704","label":"austinol metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1900561","label":"dehydroaustinol metabolic process GO:1900561","weight":24}},{"data":{"id":"GO:0019748","label":"dehydroaustinol metabolic process GO:0019748","weight":24}},{"data":{"id":"GO:1900561","label":"dehydroaustinol metabolic process GO:1900561","weight":24}},{"data":{"id":"GO:0071704","label":"dehydroaustinol metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1900585","label":"arugosin metabolic process GO:1900585","weight":24}},{"data":{"id":"GO:0019748","label":"arugosin metabolic process GO:0019748","weight":24}},{"data":{"id":"GO:1900585","label":"arugosin metabolic process GO:1900585","weight":24}},{"data":{"id":"GO:0071704","label":"arugosin metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1900619","label":"acetate ester metabolic process GO:1900619","weight":24}},{"data":{"id":"GO:0071704","label":"acetate ester metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901055","label":"trimethylenediamine metabolic process GO:1901055","weight":24}},{"data":{"id":"GO:0006807","label":"trimethylenediamine metabolic process GO:0006807","weight":24}},{"data":{"id":"GO:1901055","label":"trimethylenediamine metabolic process GO:1901055","weight":24}},{"data":{"id":"GO:0071704","label":"trimethylenediamine metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901125","label":"candicidin metabolic process GO:1901125","weight":24}},{"data":{"id":"GO:0071704","label":"candicidin metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901135","label":"carbohydrate derivative metabolic process GO:1901135","weight":24}},{"data":{"id":"GO:0071704","label":"carbohydrate derivative metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901360","label":"organic cyclic compound metabolic process GO:1901360","weight":24}},{"data":{"id":"GO:0071704","label":"organic cyclic compound metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901440","label":"poly(hydroxyalkanoate) metabolic process GO:1901440","weight":24}},{"data":{"id":"GO:0071704","label":"poly(hydroxyalkanoate) metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901564","label":"organonitrogen compound metabolic process GO:1901564","weight":24}},{"data":{"id":"GO:0006807","label":"organonitrogen compound metabolic process GO:0006807","weight":24}},{"data":{"id":"GO:1901564","label":"organonitrogen compound metabolic process GO:1901564","weight":24}},{"data":{"id":"GO:0071704","label":"organonitrogen compound metabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901575","label":"organic substance catabolic process GO:1901575","weight":24}},{"data":{"id":"GO:0009056","label":"organic substance catabolic process GO:0009056","weight":24}},{"data":{"id":"GO:1901575","label":"organic substance catabolic process GO:1901575","weight":24}},{"data":{"id":"GO:0071704","label":"organic substance catabolic process GO:0071704","weight":24}},{"data":{"id":"GO:1901576","label":"organic substance biosynthetic process GO:1901576","weight":24}},{"data":{"id":"GO:0009058","label":"organic substance biosynthetic process GO:0009058","weight":24}},{"data":{"id":"GO:1901576","label":"organic substance biosynthetic process GO:1901576","weight":24}},{"data":{"id":"GO:0071704","label":"organic substance biosynthetic process GO:0071704","weight":24}},{"data":{"id":"GO:1901615","label":"organic hydroxy compound metabolic process GO:1901615","weight":24}},{"data":{"id":"GO:0071704","label":"organic hydroxy compound metabolic process GO:0071704","weight":24}},{"data":{"id":"e0","source":"GO:0005975","target":"GO:0044238","weight":0.3}},{"data":{"id":"e1","source":"GO:0005975","target":"GO:0071704","weight":0.3}},{"data":{"id":"e2","source":"GO:0006081","target":"GO:0044237","weight":0.3}},{"data":{"id":"e3","source":"GO:0006081","target":"GO:0071704","weight":0.3}},{"data":{"id":"e4","source":"GO:0006082","target":"GO:0044237","weight":0.3}},{"data":{"id":"e5","source":"GO:0006082","target":"GO:0044281","weight":0.3}},{"data":{"id":"e6","source":"GO:0006082","target":"GO:0071704","weight":0.3}},{"data":{"id":"e7","source":"GO:0006629","target":"GO:0044238","weight":0.3}},{"data":{"id":"e8","source":"GO:0006629","target":"GO:0071704","weight":0.3}},{"data":{"id":"e9","source":"GO:0006749","target":"GO:0071704","weight":0.3}},{"data":{"id":"e10","source":"GO:0009812","target":"GO:0071704","weight":0.3}},{"data":{"id":"e11","source":"GO:0015977","target":"GO:0071704","weight":0.3}},{"data":{"id":"e12","source":"GO:0018904","target":"GO:0044281","weight":0.3}},{"data":{"id":"e13","source":"GO:0018904","target":"GO:0071704","weight":0.3}},{"data":{"id":"e14","source":"GO:0018907","target":"GO:0006790","weight":0.3}},{"data":{"id":"e15","source":"GO:0018907","target":"GO:0071704","weight":0.3}},{"data":{"id":"e16","source":"GO:0018937","target":"GO:0006805","weight":0.3}},{"data":{"id":"e17","source":"GO:0018937","target":"GO:0006807","weight":0.3}},{"data":{"id":"e18","source":"GO:0018937","target":"GO:0071704","weight":0.3}},{"data":{"id":"e19","source":"GO:0018938","target":"GO:0006805","weight":0.3}},{"data":{"id":"e20","source":"GO:0018938","target":"GO:0034641","weight":0.3}},{"data":{"id":"e21","source":"GO:0018938","target":"GO:0071704","weight":0.3}},{"data":{"id":"e22","source":"GO:0018942","target":"GO:0044237","weight":0.3}},{"data":{"id":"e23","source":"GO:0018942","target":"GO:0071704","weight":0.3}},{"data":{"id":"e24","source":"GO:0018945","target":"GO:0006805","weight":0.3}},{"data":{"id":"e25","source":"GO:0018945","target":"GO:0071704","weight":0.3}},{"data":{"id":"e26","source":"GO:0018954","target":"GO:0006805","weight":0.3}},{"data":{"id":"e27","source":"GO:0018954","target":"GO:0006807","weight":0.3}},{"data":{"id":"e28","source":"GO:0018954","target":"GO:0071704","weight":0.3}},{"data":{"id":"e29","source":"GO:0019330","target":"GO:0006807","weight":0.3}},{"data":{"id":"e30","source":"GO:0019330","target":"GO:0071704","weight":0.3}},{"data":{"id":"e31","source":"GO:0019637","target":"GO:0006793","weight":0.3}},{"data":{"id":"e32","source":"GO:0019637","target":"GO:0071704","weight":0.3}},{"data":{"id":"e33","source":"GO:0030638","target":"GO:0019748","weight":0.3}},{"data":{"id":"e34","source":"GO:0030638","target":"GO:0071704","weight":0.3}},{"data":{"id":"e35","source":"GO:0035383","target":"GO:0006790","weight":0.3}},{"data":{"id":"e36","source":"GO:0035383","target":"GO:0071704","weight":0.3}},{"data":{"id":"e37","source":"GO:0042180","target":"GO:0044237","weight":0.3}},{"data":{"id":"e38","source":"GO:0042180","target":"GO:0044281","weight":0.3}},{"data":{"id":"e39","source":"GO:0042180","target":"GO:0071704","weight":0.3}},{"data":{"id":"e40","source":"GO:0042197","target":"GO:0006805","weight":0.3}},{"data":{"id":"e41","source":"GO:0042197","target":"GO:0071704","weight":0.3}},{"data":{"id":"e42","source":"GO:0042620","target":"GO:0071704","weight":0.3}},{"data":{"id":"e43","source":"GO:0042743","target":"GO:0071704","weight":0.3}},{"data":{"id":"e44","source":"GO:0042743","target":"GO:0072593","weight":0.3}},{"data":{"id":"e45","source":"GO:0043170","target":"GO:0071704","weight":0.3}},{"data":{"id":"e46","source":"GO:0046500","target":"GO:0006790","weight":0.3}},{"data":{"id":"e47","source":"GO:0046500","target":"GO:0071704","weight":0.3}},{"data":{"id":"e48","source":"GO:0050993","target":"GO:0006793","weight":0.3}},{"data":{"id":"e49","source":"GO:0050993","target":"GO:0071704","weight":0.3}},{"data":{"id":"e50","source":"GO:0071704","target":"GO:0008152","weight":0.3}},{"data":{"id":"e51","source":"GO:0120246","target":"GO:0071704","weight":0.3}},{"data":{"id":"e52","source":"GO:0120252","target":"GO:0071704","weight":0.3}},{"data":{"id":"e53","source":"GO:0120254","target":"GO:0071704","weight":0.3}},{"data":{"id":"e54","source":"GO:1900558","target":"GO:0071704","weight":0.3}},{"data":{"id":"e55","source":"GO:1900561","target":"GO:0019748","weight":0.3}},{"data":{"id":"e56","source":"GO:1900561","target":"GO:0071704","weight":0.3}},{"data":{"id":"e57","source":"GO:1900585","target":"GO:0019748","weight":0.3}},{"data":{"id":"e58","source":"GO:1900585","target":"GO:0071704","weight":0.3}},{"data":{"id":"e59","source":"GO:1900619","target":"GO:0071704","weight":0.3}},{"data":{"id":"e60","source":"GO:1901055","target":"GO:0006807","weight":0.3}},{"data":{"id":"e61","source":"GO:1901055","target":"GO:0071704","weight":0.3}},{"data":{"id":"e62","source":"GO:1901125","target":"GO:0071704","weight":0.3}},{"data":{"id":"e63","source":"GO:1901135","target":"GO:0071704","weight":0.3}},{"data":{"id":"e64","source":"GO:1901360","target":"GO:0071704","weight":0.3}},{"data":{"id":"e65","source":"GO:1901440","target":"GO:0071704","weight":0.3}},{"data":{"id":"e66","source":"GO:1901564","target":"GO:0006807","weight":0.3}},{"data":{"id":"e67","source":"GO:1901564","target":"GO:0071704","weight":0.3}},{"data":{"id":"e68","source":"GO:1901575","target":"GO:0009056","weight":0.3}},{"data":{"id":"e69","source":"GO:1901575","target":"GO:0071704","weight":0.3}},{"data":{"id":"e70","source":"GO:1901576","target":"GO:0009058","weight":0.3}},{"data":{"id":"e71","source":"GO:1901576","target":"GO:0071704","weight":0.3}},{"data":{"id":"e72","source":"GO:1901615","target":"GO:0071704","weight":0.3}}],"style":[{"selector":"node","style":{"shape":"circle","content":"data(label)","border-color":"#A00975"}},{"selector":"edge","style":{"line-color":"#3D1244","curve-style":"bezier","target-arrow-shape":"triangle"}}],"layout":{"name":"breadthfirst"}}

            var cy = cytoscape( graphdata );

            cy.userZoomingEnabled( false );
            
});
            };
            if ((typeof(requirejs) !==  typeof(Function)) || (typeof(requirejs.config) !== typeof(Function))) {
                var script = document.createElement("script");
                script.setAttribute("src", "https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js");
                script.onload = function(){
                    renderCyjs_d756597165f44fed9cebeea2b212d80d();
                };
                document.getElementsByTagName("head")[0].appendChild(script);
            }
            else {
                renderCyjs_d756597165f44fed9cebeea2b212d80d();
            }
</script>
</body>
</html>


### Task 6
- create the same network as above with the `ms` ontology (OBO format) and subbranch `MS:1000031`


In [ ]:
let msRawData = 
    getRawString @"https://raw.githubusercontent.com/HUPO-PSI/psi-ms-CV/master/psi-ms.obo" 
 
let ms = 
    BioFSharp.IO.Obo.parseOboTerms false msRawData 
    |> Seq.toArray                                 

let msElements = getElements ms "MS:1000031"       

let msVertices = getCytoVertices msElements        
let msEdges = getCytoEdges msElements              

cytoGraph msVertices msEdges
|> CyGraph.withSize (1300,1000)

<html>

<head>

<script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min.js"></script>

</head>

<body> <style>#e29905fd9e2 { width: 1300px; height: 1000px; display: block } </style>

<div id="e29905fd9e2"></div>

<script type="text/javascript">



            var renderCyjs_92ed709a34b144a7a9df589dbe8efba7 = function() {

            var fsharpCyjsRequire = requirejs.config({context:'fsharp-cyjs',paths:{cyjs:'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min'}}) || require;

            fsharpCyjsRequire(['cyjs'], function(Cyjs) {



            var graphdata = {"container":document.getElementById('e29905fd9e2'),"elements":[{"data":{"id":"MS:1000031","label":"instrument model MS:1000031","weight":24}},{"data":{"id":"part_of","label":"instrument model part_of","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000122","label":"Bruker Daltonics instrument model MS:1000122","weight":24}},{"data":{"id":"MS:1000031","label":"Bruker Daltonics instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000124","label":"Shimadzu instrument model MS:1000124","weight":24}},{"data":{"id":"MS:1000031","label":"Shimadzu instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000126","label":"Waters instrument model MS:1000126","weight":24}},{"data":{"id":"MS:1000031","label":"Waters instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000483","label":"Thermo Fisher Scientific instrument model MS:1000483","weight":24}},{"data":{"id":"MS:1000031","label":"Thermo Fisher Scientific instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000488","label":"Hitachi instrument model MS:1000488","weight":24}},{"data":{"id":"MS:1000031","label":"Hitachi instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000489","label":"Varian instrument model MS:1000489","weight":24}},{"data":{"id":"MS:1000031","label":"Varian instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000490","label":"Agilent instrument model MS:1000490","weight":24}},{"data":{"id":"MS:1000031","label":"Agilent instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000491","label":"Dionex instrument model MS:1000491","weight":24}},{"data":{"id":"MS:1000031","label":"Dionex instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000495","label":"Applied Biosystems instrument model MS:1000495","weight":24}},{"data":{"id":"MS:1000031","label":"Applied Biosystems instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1001800","label":"LECO instrument model MS:1001800","weight":24}},{"data":{"id":"MS:1000031","label":"LECO instrument model MS:1000031","weight":24}},{"data":{"id":"e0","source":"MS:1000031","target":"part_of","weight":0.3}},{"data":{"id":"e1","source":"MS:1000121","target":"MS:1000031","weight":0.3}},{"data":{"id":"e2","source":"MS:1000122","target":"MS:1000031","weight":0.3}},{"data":{"id":"e3","source":"MS:1000124","target":"MS:1000031","weight":0.3}},{"data":{"id":"e4","source":"MS:1000126","target":"MS:1000031","weight":0.3}},{"data":{"id":"e5","source":"MS:1000483","target":"MS:1000031","weight":0.3}},{"data":{"id":"e6","source":"MS:1000488","target":"MS:1000031","weight":0.3}},{"data":{"id":"e7","source":"MS:1000489","target":"MS:1000031","weight":0.3}},{"data":{"id":"e8","source":"MS:1000490","target":"MS:1000031","weight":0.3}},{"data":{"id":"e9","source":"MS:1000491","target":"MS:1000031","weight":0.3}},{"data":{"id":"e10","source":"MS:1000495","target":"MS:1000031","weight":0.3}},{"data":{"id":"e11","source":"MS:1001800","target":"MS:1000031","weight":0.3}}],"style":[{"selector":"node","style":{"shape":"circle","content":"data(label)","border-color":"#A00975"}},{"selector":"edge","style":{"line-color":"#3D1244","curve-style":"bezier","target-arrow-shape":"triangle"}}],"layout":{"name":"breadthfirst"}}

            var cy = cytoscape( graphdata );

            cy.userZoomingEnabled( false );

            

});

            };

            if ((typeof(requirejs) !==  typeof(Function)) || (typeof(requirejs.config) !== typeof(Function))) {

                var script = document.createElement("script");

                script.setAttribute("src", "https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js");

                script.onload = function(){

                    renderCyjs_92ed709a34b144a7a9df589dbe8efba7();

                };

                document.getElementsByTagName("head")[0].appendChild(script);

            }

            else {

                renderCyjs_92ed709a34b144a7a9df589dbe8efba7();

            }

</script>

 </body>

</html>

### Task 7
- expand the graph using elements linked to `MS:1000121`
- triple elements can be appended by `Seq.append seq1 seq2`
- check out the graph

In [ ]:
let msElements2 = getElements ms "MS:1000121"

let msVertices2 = getCytoVertices (Seq.append msElements msElements2)
let msEdges2 = getCytoEdges (Seq.append msElements msElements2)

cytoGraph msVertices2 msEdges2
|> CyGraph.withSize (1300,1000)

<html>

<head>

<script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min.js"></script>

</head>

<body> <style>#e03fc6d45ba { width: 1300px; height: 1000px; display: block } </style>

<div id="e03fc6d45ba"></div>

<script type="text/javascript">



            var renderCyjs_332a9a43714641d1b5a4e485ba54749b = function() {

            var fsharpCyjsRequire = requirejs.config({context:'fsharp-cyjs',paths:{cyjs:'https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.18.0/cytoscape.min'}}) || require;

            fsharpCyjsRequire(['cyjs'], function(Cyjs) {



            var graphdata = {"container":document.getElementById('e03fc6d45ba'),"elements":[{"data":{"id":"MS:1000031","label":"instrument model MS:1000031","weight":24}},{"data":{"id":"part_of","label":"instrument model part_of","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000122","label":"Bruker Daltonics instrument model MS:1000122","weight":24}},{"data":{"id":"MS:1000031","label":"Bruker Daltonics instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000124","label":"Shimadzu instrument model MS:1000124","weight":24}},{"data":{"id":"MS:1000031","label":"Shimadzu instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000126","label":"Waters instrument model MS:1000126","weight":24}},{"data":{"id":"MS:1000031","label":"Waters instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000483","label":"Thermo Fisher Scientific instrument model MS:1000483","weight":24}},{"data":{"id":"MS:1000031","label":"Thermo Fisher Scientific instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000488","label":"Hitachi instrument model MS:1000488","weight":24}},{"data":{"id":"MS:1000031","label":"Hitachi instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000489","label":"Varian instrument model MS:1000489","weight":24}},{"data":{"id":"MS:1000031","label":"Varian instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000490","label":"Agilent instrument model MS:1000490","weight":24}},{"data":{"id":"MS:1000031","label":"Agilent instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000491","label":"Dionex instrument model MS:1000491","weight":24}},{"data":{"id":"MS:1000031","label":"Dionex instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000495","label":"Applied Biosystems instrument model MS:1000495","weight":24}},{"data":{"id":"MS:1000031","label":"Applied Biosystems instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1001800","label":"LECO instrument model MS:1001800","weight":24}},{"data":{"id":"MS:1000031","label":"LECO instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000121","label":"SCIEX instrument model MS:1000121","weight":24}},{"data":{"id":"MS:1000031","label":"SCIEX instrument model MS:1000031","weight":24}},{"data":{"id":"MS:1000139","label":"4000 QTRAP MS:1000139","weight":24}},{"data":{"id":"MS:1000121","label":"4000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000143","label":"API 150EX MS:1000143","weight":24}},{"data":{"id":"MS:1000121","label":"API 150EX MS:1000121","weight":24}},{"data":{"id":"MS:1000144","label":"API 150EX Prep MS:1000144","weight":24}},{"data":{"id":"MS:1000121","label":"API 150EX Prep MS:1000121","weight":24}},{"data":{"id":"MS:1000145","label":"API 2000 MS:1000145","weight":24}},{"data":{"id":"MS:1000121","label":"API 2000 MS:1000121","weight":24}},{"data":{"id":"MS:1000146","label":"API 3000 MS:1000146","weight":24}},{"data":{"id":"MS:1000121","label":"API 3000 MS:1000121","weight":24}},{"data":{"id":"MS:1000147","label":"API 4000 MS:1000147","weight":24}},{"data":{"id":"MS:1000121","label":"API 4000 MS:1000121","weight":24}},{"data":{"id":"MS:1000186","label":"proteomics solution 1 MS:1000186","weight":24}},{"data":{"id":"MS:1000121","label":"proteomics solution 1 MS:1000121","weight":24}},{"data":{"id":"MS:1000187","label":"Q TRAP MS:1000187","weight":24}},{"data":{"id":"MS:1000121","label":"Q TRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000190","label":"QSTAR MS:1000190","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR MS:1000121","weight":24}},{"data":{"id":"MS:1000194","label":"SymBiot I MS:1000194","weight":24}},{"data":{"id":"MS:1000121","label":"SymBiot I MS:1000121","weight":24}},{"data":{"id":"MS:1000195","label":"SymBiot XVI MS:1000195","weight":24}},{"data":{"id":"MS:1000121","label":"SymBiot XVI MS:1000121","weight":24}},{"data":{"id":"MS:1000651","label":"3200 QTRAP MS:1000651","weight":24}},{"data":{"id":"MS:1000121","label":"3200 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000652","label":"4800 Plus MALDI TOF/TOF MS:1000652","weight":24}},{"data":{"id":"MS:1000121","label":"4800 Plus MALDI TOF/TOF MS:1000121","weight":24}},{"data":{"id":"MS:1000653","label":"API 3200 MS:1000653","weight":24}},{"data":{"id":"MS:1000121","label":"API 3200 MS:1000121","weight":24}},{"data":{"id":"MS:1000654","label":"API 5000 MS:1000654","weight":24}},{"data":{"id":"MS:1000121","label":"API 5000 MS:1000121","weight":24}},{"data":{"id":"MS:1000655","label":"QSTAR Elite MS:1000655","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR Elite MS:1000121","weight":24}},{"data":{"id":"MS:1000656","label":"QSTAR Pulsar MS:1000656","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR Pulsar MS:1000121","weight":24}},{"data":{"id":"MS:1000657","label":"QSTAR XL MS:1000657","weight":24}},{"data":{"id":"MS:1000121","label":"QSTAR XL MS:1000121","weight":24}},{"data":{"id":"MS:1000870","label":"4000 QTRAP MS:1000870","weight":24}},{"data":{"id":"MS:1000121","label":"4000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1000931","label":"QTRAP 5500 MS:1000931","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 5500 MS:1000121","weight":24}},{"data":{"id":"MS:1000932","label":"TripleTOF 5600 MS:1000932","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 5600 MS:1000121","weight":24}},{"data":{"id":"MS:1001482","label":"5800 TOF/TOF MS:1001482","weight":24}},{"data":{"id":"MS:1000121","label":"5800 TOF/TOF MS:1000121","weight":24}},{"data":{"id":"MS:1002533","label":"TripleTOF 6600 MS:1002533","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 6600 MS:1000121","weight":24}},{"data":{"id":"MS:1002577","label":"2000 QTRAP MS:1002577","weight":24}},{"data":{"id":"MS:1000121","label":"2000 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002578","label":"2500 QTRAP MS:1002578","weight":24}},{"data":{"id":"MS:1000121","label":"2500 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002579","label":"3500 QTRAP MS:1002579","weight":24}},{"data":{"id":"MS:1000121","label":"3500 QTRAP MS:1000121","weight":24}},{"data":{"id":"MS:1002580","label":"QTRAP 4500 MS:1002580","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 4500 MS:1000121","weight":24}},{"data":{"id":"MS:1002581","label":"QTRAP 6500 MS:1002581","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 6500 MS:1000121","weight":24}},{"data":{"id":"MS:1002582","label":"QTRAP 6500+ MS:1002582","weight":24}},{"data":{"id":"MS:1000121","label":"QTRAP 6500+ MS:1000121","weight":24}},{"data":{"id":"MS:1002583","label":"TripleTOF 4600 MS:1002583","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 4600 MS:1000121","weight":24}},{"data":{"id":"MS:1002584","label":"TripleTOF 5600+ MS:1002584","weight":24}},{"data":{"id":"MS:1000121","label":"TripleTOF 5600+ MS:1000121","weight":24}},{"data":{"id":"MS:1002585","label":"API 100 MS:1002585","weight":24}},{"data":{"id":"MS:1000121","label":"API 100 MS:1000121","weight":24}},{"data":{"id":"MS:1002586","label":"API 100LC MS:1002586","weight":24}},{"data":{"id":"MS:1000121","label":"API 100LC MS:1000121","weight":24}},{"data":{"id":"MS:1002587","label":"API 165 MS:1002587","weight":24}},{"data":{"id":"MS:1000121","label":"API 165 MS:1000121","weight":24}},{"data":{"id":"MS:1002588","label":"API 300 MS:1002588","weight":24}},{"data":{"id":"MS:1000121","label":"API 300 MS:1000121","weight":24}},{"data":{"id":"MS:1002589","label":"API 350 MS:1002589","weight":24}},{"data":{"id":"MS:1000121","label":"API 350 MS:1000121","weight":24}},{"data":{"id":"MS:1002590","label":"API 365 MS:1002590","weight":24}},{"data":{"id":"MS:1000121","label":"API 365 MS:1000121","weight":24}},{"data":{"id":"MS:1002591","label":"Triple Quad 3500 MS:1002591","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 3500 MS:1000121","weight":24}},{"data":{"id":"MS:1002592","label":"Triple Quad 4500 MS:1002592","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 4500 MS:1000121","weight":24}},{"data":{"id":"MS:1002593","label":"Triple Quad 5500 MS:1002593","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 5500 MS:1000121","weight":24}},{"data":{"id":"MS:1002594","label":"Triple Quad 6500 MS:1002594","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 6500 MS:1000121","weight":24}},{"data":{"id":"MS:1002595","label":"Triple Quad 6500+ MS:1002595","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 6500+ MS:1000121","weight":24}},{"data":{"id":"MS:1002674","label":"X500R QTOF MS:1002674","weight":24}},{"data":{"id":"MS:1000121","label":"X500R QTOF MS:1000121","weight":24}},{"data":{"id":"MS:1003144","label":"Triple Quad 7500 MS:1003144","weight":24}},{"data":{"id":"MS:1000121","label":"Triple Quad 7500 MS:1000121","weight":24}},{"data":{"id":"MS:1003293","label":"ZenoTOF 7600 MS:1003293","weight":24}},{"data":{"id":"MS:1000121","label":"ZenoTOF 7600 MS:1000121","weight":24}},{"data":{"id":"e0","source":"MS:1000031","target":"part_of","weight":0.3}},{"data":{"id":"e1","source":"MS:1000121","target":"MS:1000031","weight":0.3}},{"data":{"id":"e2","source":"MS:1000122","target":"MS:1000031","weight":0.3}},{"data":{"id":"e3","source":"MS:1000124","target":"MS:1000031","weight":0.3}},{"data":{"id":"e4","source":"MS:1000126","target":"MS:1000031","weight":0.3}},{"data":{"id":"e5","source":"MS:1000483","target":"MS:1000031","weight":0.3}},{"data":{"id":"e6","source":"MS:1000488","target":"MS:1000031","weight":0.3}},{"data":{"id":"e7","source":"MS:1000489","target":"MS:1000031","weight":0.3}},{"data":{"id":"e8","source":"MS:1000490","target":"MS:1000031","weight":0.3}},{"data":{"id":"e9","source":"MS:1000491","target":"MS:1000031","weight":0.3}},{"data":{"id":"e10","source":"MS:1000495","target":"MS:1000031","weight":0.3}},{"data":{"id":"e11","source":"MS:1001800","target":"MS:1000031","weight":0.3}},{"data":{"id":"e12","source":"MS:1000139","target":"MS:1000121","weight":0.3}},{"data":{"id":"e13","source":"MS:1000143","target":"MS:1000121","weight":0.3}},{"data":{"id":"e14","source":"MS:1000144","target":"MS:1000121","weight":0.3}},{"data":{"id":"e15","source":"MS:1000145","target":"MS:1000121","weight":0.3}},{"data":{"id":"e16","source":"MS:1000146","target":"MS:1000121","weight":0.3}},{"data":{"id":"e17","source":"MS:1000147","target":"MS:1000121","weight":0.3}},{"data":{"id":"e18","source":"MS:1000186","target":"MS:1000121","weight":0.3}},{"data":{"id":"e19","source":"MS:1000187","target":"MS:1000121","weight":0.3}},{"data":{"id":"e20","source":"MS:1000190","target":"MS:1000121","weight":0.3}},{"data":{"id":"e21","source":"MS:1000194","target":"MS:1000121","weight":0.3}},{"data":{"id":"e22","source":"MS:1000195","target":"MS:1000121","weight":0.3}},{"data":{"id":"e23","source":"MS:1000651","target":"MS:1000121","weight":0.3}},{"data":{"id":"e24","source":"MS:1000652","target":"MS:1000121","weight":0.3}},{"data":{"id":"e25","source":"MS:1000653","target":"MS:1000121","weight":0.3}},{"data":{"id":"e26","source":"MS:1000654","target":"MS:1000121","weight":0.3}},{"data":{"id":"e27","source":"MS:1000655","target":"MS:1000121","weight":0.3}},{"data":{"id":"e28","source":"MS:1000656","target":"MS:1000121","weight":0.3}},{"data":{"id":"e29","source":"MS:1000657","target":"MS:1000121","weight":0.3}},{"data":{"id":"e30","source":"MS:1000870","target":"MS:1000121","weight":0.3}},{"data":{"id":"e31","source":"MS:1000931","target":"MS:1000121","weight":0.3}},{"data":{"id":"e32","source":"MS:1000932","target":"MS:1000121","weight":0.3}},{"data":{"id":"e33","source":"MS:1001482","target":"MS:1000121","weight":0.3}},{"data":{"id":"e34","source":"MS:1002533","target":"MS:1000121","weight":0.3}},{"data":{"id":"e35","source":"MS:1002577","target":"MS:1000121","weight":0.3}},{"data":{"id":"e36","source":"MS:1002578","target":"MS:1000121","weight":0.3}},{"data":{"id":"e37","source":"MS:1002579","target":"MS:1000121","weight":0.3}},{"data":{"id":"e38","source":"MS:1002580","target":"MS:1000121","weight":0.3}},{"data":{"id":"e39","source":"MS:1002581","target":"MS:1000121","weight":0.3}},{"data":{"id":"e40","source":"MS:1002582","target":"MS:1000121","weight":0.3}},{"data":{"id":"e41","source":"MS:1002583","target":"MS:1000121","weight":0.3}},{"data":{"id":"e42","source":"MS:1002584","target":"MS:1000121","weight":0.3}},{"data":{"id":"e43","source":"MS:1002585","target":"MS:1000121","weight":0.3}},{"data":{"id":"e44","source":"MS:1002586","target":"MS:1000121","weight":0.3}},{"data":{"id":"e45","source":"MS:1002587","target":"MS:1000121","weight":0.3}},{"data":{"id":"e46","source":"MS:1002588","target":"MS:1000121","weight":0.3}},{"data":{"id":"e47","source":"MS:1002589","target":"MS:1000121","weight":0.3}},{"data":{"id":"e48","source":"MS:1002590","target":"MS:1000121","weight":0.3}},{"data":{"id":"e49","source":"MS:1002591","target":"MS:1000121","weight":0.3}},{"data":{"id":"e50","source":"MS:1002592","target":"MS:1000121","weight":0.3}},{"data":{"id":"e51","source":"MS:1002593","target":"MS:1000121","weight":0.3}},{"data":{"id":"e52","source":"MS:1002594","target":"MS:1000121","weight":0.3}},{"data":{"id":"e53","source":"MS:1002595","target":"MS:1000121","weight":0.3}},{"data":{"id":"e54","source":"MS:1002674","target":"MS:1000121","weight":0.3}},{"data":{"id":"e55","source":"MS:1003144","target":"MS:1000121","weight":0.3}},{"data":{"id":"e56","source":"MS:1003293","target":"MS:1000121","weight":0.3}}],"style":[{"selector":"node","style":{"shape":"circle","content":"data(label)","border-color":"#A00975"}},{"selector":"edge","style":{"line-color":"#3D1244","curve-style":"bezier","target-arrow-shape":"triangle"}}],"layout":{"name":"breadthfirst"}}

            var cy = cytoscape( graphdata );

            cy.userZoomingEnabled( false );

            

});

            };

            if ((typeof(requirejs) !==  typeof(Function)) || (typeof(requirejs.config) !== typeof(Function))) {

                var script = document.createElement("script");

                script.setAttribute("src", "https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js");

                script.onload = function(){

                    renderCyjs_332a9a43714641d1b5a4e485ba54749b();

                };

                document.getElementsByTagName("head")[0].appendChild(script);

            }

            else {

                renderCyjs_332a9a43714641d1b5a4e485ba54749b();

            }

</script>

 </body>

</html>